# Export image function notebook


## Notebook for Exporting Images from NICFI and Sentinel Datasource

### Exporting Rules:
- Target shapefile: 1823_ADRSM
- Important fields: `DEN_BOT`, `AREA_HA`, `Index`, `SHAPE@`
- Export only if `AREA_HA` > 10
- NICFI datasource imported via GEE in Python notebook
- Sentinel-2 data could be imported through several channels; this solution utilizes ArcGIS Pro Living Atlas (AWS host)
- Dynamic scale properties:
  - If size < 10ha, image will scale at 8 times 
  - If 10ha < size < 50ha, image will scale at 5 times
  - If 50ha < shape size < 100ha, image will scale at 2 times
  - If shape size > 100ha, image will scale at 1.3 times

- Five Years and 60 images for each dumpsite using NICFI  

### Usage:  
1. Each block contains the function description
2. NICFI:
   - Get NICFI
   - Add NICFI
   - Determine the date range (check the new update)
   - Get Project, layout, map, mapview, mapframe
   - Processing other Exporting 
3. Sentinel: 
   - Add sentinel data from Living Atlas (or other resource option)
4. Export multiple date's images (5 years x 12 months) of each shape

### Environment:
- ArcGIS Pro 3.2.2
- Python 3.8.19


## Basic Function
Run basic before export.

In [1]:
# lib import 

import arcpy
# check the shapefile return selected count of feature class
print(arcpy.GetCount_management("1823_ADRSM")) # if selected 1 it will return 1

print("Import, and load shape")

1823
Import, and load shape


In [2]:
# Get nicif

# add gee into arcgispro
import ee
import geemap

# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize(project='ee-qinheyi')

# init the map
m=geemap.Map()

nicfi = ee.ImageCollection('projects/planet-nicfi/assets/basemaps/americas')

# check the date range of image collection
def getDateRange(imageCollection:ee.ImageCollection):
    dateRange=imageCollection.aggregate_array('system:time_start')
    date_list=[ee.Date(date).format().getInfo() for date in dateRange.getInfo()]
    return date_list

# imageRange=getDateRange(nicfi) # because of the server-side function.

baseMap=nicfi.filter(ee.Filter.date('2024-02-01','2025-01-11')).sort('system:time_start', False).first()

print("nicfi datespan:",getDateRange(nicfi))
print("Basemap loaded")

nicfi datespan: ['2015-12-01T00:00:00', '2016-06-01T00:00:00', '2016-12-01T00:00:00', '2017-06-01T00:00:00', '2017-12-01T00:00:00', '2018-06-01T00:00:00', '2018-12-01T00:00:00', '2019-06-01T00:00:00', '2019-12-01T00:00:00', '2020-06-01T00:00:00', '2020-09-01T00:00:00', '2020-10-01T00:00:00', '2020-11-01T00:00:00', '2020-12-01T00:00:00', '2021-01-01T00:00:00', '2021-02-01T00:00:00', '2021-03-01T00:00:00', '2021-04-01T00:00:00', '2021-05-01T00:00:00', '2021-06-01T00:00:00', '2021-07-01T00:00:00', '2021-08-01T00:00:00', '2021-09-01T00:00:00', '2021-10-01T00:00:00', '2021-11-01T00:00:00', '2021-12-01T00:00:00', '2022-01-01T00:00:00', '2022-02-01T00:00:00', '2022-03-01T00:00:00', '2022-04-01T00:00:00', '2022-05-01T00:00:00', '2022-06-01T00:00:00', '2022-07-01T00:00:00', '2022-08-01T00:00:00', '2022-09-01T00:00:00', '2022-10-01T00:00:00', '2022-11-01T00:00:00', '2022-12-01T00:00:00', '2023-01-01T00:00:00', '2023-02-01T00:00:00', '2023-03-01T00:00:00', '2023-04-01T00:00:00', '2023-05-01T00:00

In [3]:
# Add nicif

# m.centerObject(baseMap,4)
# vis={'bands':['R','G','B'],'min':64,'max':5454,'gamma':1.8}  # original vis
vis={'bands':['R','G','B'],'min':500,'max':3500,'gamma':2.0}
m.addLayer(baseMap,vis,'nicfi')
m.height='1600px'
display(m)
print("Map show complete")

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

Map show complete


In [4]:
# Get Project, layout, map, mapview, mapframe
# Before using this code , you should open and load both layout window and Map window, or it will erro: mapview is None

current_project=arcpy.mp.ArcGISProject("CURRENT")
layout=current_project.listLayouts()[1]  # 0 is the dataframe drive layout(text,area) # 1 is the 
amap=current_project.listMaps()[0]  
mv=current_project.activeView    #mv=amap.defaultView
mf= layout.listElements("MAPFRAME_ELEMENT")[0]  #get the mapframe in the layout

print("Current layout name:",layout.name)
print("Mapview.extent: ", mv.camera.getExtent())
print("Mapname: ",amap.name)
print("Mfname: ",mf.name)

Current layout name: JpegExport
Mapview.extent:  -8847867.95292556 -581654.623630882 -8846359.73645093 -579802.39115925 NaN NaN NaN NaN
Mapname:  Map
Mfname:  Map Frame


## Exporting func by certain time

In [5]:
# this function is for export the images 
# it can be call onetime or iterate all in different year and month datasource set
import os

#only based on the shape "1823_ADRSM" and the 
fc = "1823_ADRSM"
fields = ['DEN_BOT','AREA_HA','Index',"SHAPE@" ]
default_path=r"C:\Users\lycaz\Desktop\output"

def export_image(time,isScale=True,path=default_path,src_label="nicfi",dpi=150):
    """
    Brief description of the function.

    Parameters:
    isScale (Bool): set for whether dynamic-scale or not 
    time (str): Description of param2.
    path(str): which folder you want to saved
    
    Returns:
    type: Description of what the function returns.
    """
    if os.path.exists(path)==False:
        print("Path does not exist.")
        return
    
    with arcpy.da.SearchCursor(fc, fields) as cursor:
        count_im=0
        for row in cursor:
            #print(u'{0}, {1},{2}'.format(row[2],row[0], row[1]))
            print(f" Processing index {row[2]} {row[0]}", end="\r")
            if row[1] > 10:
                count_im+=1
                print(f"image coount {count_im}", end="\r")
                geometry=row[3]
                mv.camera.setExtent(geometry.extent)
                amap.defaultCamera=mv.camera
                amap.defaultCamera.scale*=1.5
                mf.camera.setExtent(amap.defaultCamera.getExtent())
                if isScale:
                    if row[1]<10:
                        mf.camera.scale*=8
                    elif row[1]<20:
                        mf.camera.scale*=5
                    elif row[1]<100:
                        mf.camera.scale*=2
                    else:
                        mf.camera.scale*=1.3
                ele=layout.listElements(wildcard="Text")[0]
                ele.text=f"{row[2]}-{row[0]}-{time}-{row[1]:.2f}ha"
                ele_source=layout.listElements(wildcard="Text 1")[0]
                ele_source.text=src_label
                layout.exportToJPEG(f"{path}\\{str(row[2])}-{time}-{src_label}.jpg",resolution=dpi)
        print('\n')
        print(f"Complete current the image in: {time} ")
    
    
print(f"Exporting function ready path:{default_path}")   
    
    

Exporting function ready path:C:\Users\lycaz\Desktop\output


In [ ]:
# Call once Exporting example of one certain year based on currently tile file
# Before exporting, confirm the layout is setting well, the tile map is loaded and the shapefile is outlined.



# get the certain shape of shapefile  and iterate the shapefile
# handle the mv (mapview), do not effect the layout 
import time 
start_time = time.time()

fc = "1823_ADRSM"
fields = ['DEN_BOT','AREA_HA','Index',"SHAPE@" ]

path=r"C:\Users\lycaz\Desktop\output"    # schange it before exporting !!!!!!
time_stamp="202403"                         # check it as well 
text_datasource="Sentinal" # or "Nicif" "Sentinel"        # change it bfore export 

# call the exporting function
export_image(time_stamp,True,path,"nicfi",150)

stop_time = time.time()
elapsed_time = stop_time - start_time

print(f"Start time: {time.ctime(start_time)}")
print(f"Stop time: {time.ctime(stop_time)}")
print(f"Elapsed time: {elapsed_time:.2f} seconds")

## Exporting image by time series

### For nicfi 
The code is for nicfi which is load by google earth engine.

Using iteration load multiple layer then do the exporting function on each layer

In [7]:
# Function for exporting history images in 5x12 month from nicfi source , n shapes x 60 images
# Get 60 tile layers and add it to imageview each layer export n shape. There is no built in function to change one layer and update the layer to certain date.


# Check the times. 
# from "2017-01-01" to "2024-05-01"  to get 50 date from the collection
start_date="2017-01-01"
end_date="2024-4-01"

nicfi_collection=nicfi.filter(ee.Filter.date(start_date,end_date)).sort('system:time_start', False)
dates=nicfi_collection.aggregate_array('system:time_start')
# Convert dates to a list and label_list
dates_list = dates.getInfo()
date_lables=[]
# Iterate over the dates and print them out
print(len(dates_list))
for date in dates_list:
    date_lables.append(ee.Date(date).format("YYYY-MM-dd").getInfo())
print("Date:",date_lables)



50
Date: ['2024-03-01', '2024-02-01', '2024-01-01', '2023-12-01', '2023-11-01', '2023-10-01', '2023-09-01', '2023-08-01', '2023-07-01', '2023-06-01', '2023-05-01', '2023-04-01', '2023-03-01', '2023-02-01', '2023-01-01', '2022-12-01', '2022-11-01', '2022-10-01', '2022-09-01', '2022-08-01', '2022-07-01', '2022-06-01', '2022-05-01', '2022-04-01', '2022-03-01', '2022-02-01', '2022-01-01', '2021-12-01', '2021-11-01', '2021-10-01', '2021-09-01', '2021-08-01', '2021-07-01', '2021-06-01', '2021-05-01', '2021-04-01', '2021-03-01', '2021-02-01', '2021-01-01', '2020-12-01', '2020-11-01', '2020-10-01', '2020-09-01', '2020-06-01', '2019-12-01', '2019-06-01', '2018-12-01', '2018-06-01', '2017-12-01', '2017-06-01']


In [18]:
# iterate the time and export in each iteration
# this block of code is independency from the 

# find the shapefile layer and always put it to top

import time 
start_time = time.time()

target_layer_name="1823_ADRSM"
top_layer=None
for lyr in amap.listLayers():
    if lyr.name==target_layer_name:
        top_layer=lyr

def put_layer_top(ta_map=amap,ta_layer=top_layer):
    if ta_layer is not None:
        ta_map.moveLayer(amap.listLayers()[0],ta_layer,"BEFORE")
        print(f"{ta_layer}Layer moved to the top successfully.")
    else:
        print("Layer not found in the map.")

def turn_off_other_layer():
    for ly in amap.listLayers():
        if ly.name!=target_layer_name:
            ly.visible = False


export_path=r"C:\Users\lycaz\Desktop\output"
label_text="nicfi"

for i,date in enumerate(dates_list):
    month=ee.Date(date).get('month').getInfo()
    s_d=ee.Date(date).format("YYYY-MM-dd").getInfo()
    e_d=ee.Date(date).advance(1, 'month').format("YYYY-MM-dd").getInfo()
    print("Adding Date Range:",s_d,e_d)
    image=nicfi.filter(ee.Filter.date(s_d,e_d)).sort('system:time_start', False).first()
    layer_name = ee.Date(date).format("YYYY-MM-dd").getInfo()
    turn_off_other_layer() # turn off all layer
    m.addLayer(image,vis,layer_name) # add new layer
    put_layer_top(amap,top_layer) # # put the layer top
    
    # print export image start time 
    sta_time = time.time()
    print(f"Start {layer_name} exportin time: {time.ctime(sta_time)}")
    export_image(layer_name.replace("-","")[:6],True,export_path,label_text,150) #150dpi would save time for 38file/min
    sto_time = time.time()
    print(f"End {layer_name} exportin time: {time.ctime(sto_time)}")
    
    
stop_time = time.time()
elapsed_time = stop_time - start_time

print(f"Start time: {time.ctime(start_time)}")
print(f"Stop time: {time.ctime(stop_time)}")
print(f"Elapsed time: {elapsed_time:.2f} seconds") 


Adding Date Range: 2023-09-01 2023-10-01
1823_ADRSMLayer moved to the top successfully.
Start 2023-09-01 exportin time: Wed Apr 17 03:13:09 2024
 Processing index 1822 Botadero S/N (Punchana)elto Sector Qochayoqpatachuato del servicio de gestión integral de residuos solidos del distrito de Ite

Complete current the image in: 202309 
End 2023-09-01 exportin time: Wed Apr 17 03:14:51 2024
Adding Date Range: 2023-08-01 2023-09-01
1823_ADRSMLayer moved to the top successfully.
Start 2023-08-01 exportin time: Wed Apr 17 03:14:52 2024
 Processing index 1822 Botadero S/N (Punchana)elto Sector Qochayoqpatachuato del servicio de gestión integral de residuos solidos del distrito de Ite

Complete current the image in: 202308 
End 2023-08-01 exportin time: Wed Apr 17 03:16:05 2024
Adding Date Range: 2023-07-01 2023-08-01
1823_ADRSMLayer moved to the top successfully.
Start 2023-07-01 exportin time: Wed Apr 17 03:16:06 2024
 Processing index 1822 Botadero S/N (Punchana)elto Sector Qochayoqpatachuat

Adding Date Range: 2017-12-01 2018-01-01
1823_ADRSMLayer moved to the top successfully.
Start 2017-12-01 exportin time: Wed Apr 17 04:02:51 2024
 Processing index 1822 Botadero S/N (Punchana)elto Sector Qochayoqpatachuato del servicio de gestión integral de residuos solidos del distrito de Ite

Complete current the image in: 201712 
End 2017-12-01 exportin time: Wed Apr 17 04:03:58 2024
Adding Date Range: 2017-06-01 2017-07-01
1823_ADRSMLayer moved to the top successfully.
Start 2017-06-01 exportin time: Wed Apr 17 04:03:59 2024
 Processing index 1822 Botadero S/N (Punchana)elto Sector Qochayoqpatachuato del servicio de gestión integral de residuos solidos del distrito de Ite

Complete current the image in: 201706 
End 2017-06-01 exportin time: Wed Apr 17 04:05:12 2024
Start time: Wed Apr 17 03:13:08 2024
Stop time: Wed Apr 17 04:05:12 2024
Elapsed time: 3124.16 seconds


### For sentinel
Sentinel2 data is from LivingAtlas. which could using the arcgis time slide to control

Sentinel2 can be load from several different source (livingatlas,  Sentinel-2 on AWS collections [ArcGISserver](https://sentinel.arcgis.com/arcgis/rest/services/Sentinel2/ImageServer)...)
LivingGatlas is best performance

In [6]:
# check the currently the update of the sentinel2 layer's date :
# the result acturrly is set in the time properties in the arcgis. 
sentinel_layer=None
# get the sentinel layer
for lyr in amap.listLayers():
    if lyr.supports('TIME'):
        if lyr.name=='Sentinel2L2A':
            lyr.isTimeEnabled==True
            # get the sentinel2l2a layer with time series
            print(lyr.time.startTime, lyr.time.endTime)  # check the currently time of layer

2018-01-01 00:00:00 2024-04-17 00:00:00


In [9]:
# each month export one image for one certain shape also collect the 5x12 60 timeseriel image for each shape
import arcpy, os
import datetime
from dateutil.relativedelta import relativedelta
import time 
start_time = time.time()

# using the maptime to change whole map's time
t= mf.time
# set the timeend before time end
senti_path=r"C:\Users\lycaz\Desktop\senti"

# export the image based on the time 
t.currentTimeEnd=datetime.datetime(2024,4,1,0,0)
t.currentTimeStart=datetime.datetime(2024,3,1,0,0)
t.setTimeInclusion('INCLUDE_ONLY_START')
for i in range(0,60):
    print("Exporting time index:",i+1)
    # excut the export 
    
    export_image(t.currentTimeStart.replace("-","")[:6],True,senti_path,"sentinel2",150):
    print("Loaded: ",t.currentTimeStart ,t.currentTimeEnd)
    t.currentTimeEnd=t.timeStep(t.currentTimeEnd,-1,"months")
    t.currentTimeStart=t.timeStep(t.currentTimeStart,-1,"months")


print(f"Start time: {time.ctime(start_time)}")
print(f"Stop time: {time.ctime(stop_time)}")
print(f"Elapsed time: {elapsed_time:.2f} seconds")    
     


Loaded:  2024-03-01 00:00:00 2024-04-01 00:00:00
Loaded:  2024-02-01 00:00:00 2024-03-01 00:00:00
Loaded:  2024-01-01 00:00:00 2024-02-01 00:00:00
Loaded:  2023-12-01 00:00:00 2024-01-01 00:00:00
Loaded:  2023-11-01 00:00:00 2023-12-01 00:00:00
Loaded:  2023-10-01 00:00:00 2023-11-01 00:00:00
Loaded:  2023-09-01 00:00:00 2023-10-01 00:00:00
Loaded:  2023-08-01 00:00:00 2023-09-01 00:00:00
Loaded:  2023-07-01 00:00:00 2023-08-01 00:00:00
Loaded:  2023-06-01 00:00:00 2023-07-01 00:00:00
Loaded:  2023-05-01 00:00:00 2023-06-01 00:00:00
Loaded:  2023-04-01 00:00:00 2023-05-01 00:00:00
Loaded:  2023-03-01 00:00:00 2023-04-01 00:00:00
Loaded:  2023-02-01 00:00:00 2023-03-01 00:00:00
Loaded:  2023-01-01 00:00:00 2023-02-01 00:00:00
Loaded:  2022-12-01 00:00:00 2023-01-01 00:00:00
Loaded:  2022-11-01 00:00:00 2022-12-01 00:00:00
Loaded:  2022-10-01 00:00:00 2022-11-01 00:00:00
Loaded:  2022-09-01 00:00:00 2022-10-01 00:00:00
Loaded:  2022-08-01 00:00:00 2022-09-01 00:00:00
Loaded:  2022-07-01 

## Other function

### Center certain shape

In [12]:
# [] TODO: using gee load sentinel faild ,can not show the map

senti=ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED").filterDate('2020-01-01', '2024-01-30').filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE',20))
visualization = {
  "min": 0.0,
  "max": 0.3,
  "bands": ['B4', 'B3', 'B2']
}
m.addLayer(senti.mean(), visualization, 'senti');

In [ ]:
# mapTime using  layertime using test
t= mf.time
# set the timeend before time end
t.currentTimeEnd = datetime(2020,1,1,0,0)
t.currentTimeStart = datetime(2019,1,1,0,0)

t.setTimeInclusion('INCLUDE_ONLY_START')

print(mf.time.currentTimeStart, mf.time.currentTimeEnd)






In [31]:
# Function for timely review the imagry
# iterate all time from recently 5 years 
# show it in certain place 



# function to query certain shape return the geometrys , using index ,using size ha as query_str
def query_geometry(query_str):
    geometries = []
    # Use a search cursor to iterate over the features that match the query
    with arcpy.da.SearchCursor("1823_ADRSM", ['AREA_HA','SHAPE@'], where_clause=query_str) as cursor:
        for row in cursor:
            # Access the geometry of each feature
            geometry = row[1]
            # Add the geometry to the list
            geometries.append(geometry)
            print(row[0])
            print(type(row[1]))
    return geometries


# showcase of using query_geometry and center it 
sql_query="Index=1171"
selected_geo=query_geometry(sql_query)
if len(selected_geo)==1:
    print(selected_geo)
    print(selected_geo[0].extent.height,selected_geo[0].extent.width)
    # center to this geo
    #mv.panToExtent(selected_geo[0].extent)
    #mv.camera.setExtent(selected_geo[0].extent)
    amap.defaultCamera.setExtent(selected_geo[0].extent)
    amap.defaultCamera.scale*=1.5
    
    print(mv.camera.getExtent ())

    

    

181.24151696
<class 'arcpy.arcobjects.geometries.Polygon'>
[<Polygon object at 0x1f38ab44070[0x1f30ed3fde0]>]
0.025418119999999433 0.019702124999994908
-76.116801368 -13.3816299269999 -76.097099243 -13.3562118069999 0 0 nan nan


In [10]:
# TODO 
# change basemap
start_year=2019
end_year=2024
date_list=[]
for i in range(start_year,end_year):
    for j in range(3,13):
        if j >9:
            date_list.append(f"{i}-{j}-01")
        else:
            date_list.append(f"{i}-0{j}-01")
    date_list.append(f"{i+1}-01-01")
    date_list.append(f"{i+1}-02-01")
print(len(date_list),date_list)
 

60 ['2019-03-01', '2019-04-01', '2019-05-01', '2019-06-01', '2019-07-01', '2019-08-01', '2019-09-01', '2019-10-01', '2019-11-01', '2019-12-01', '2020-01-01', '2020-02-01', '2020-03-01', '2020-04-01', '2020-05-01', '2020-06-01', '2020-07-01', '2020-08-01', '2020-09-01', '2020-10-01', '2020-11-01', '2020-12-01', '2021-01-01', '2021-02-01', '2021-03-01', '2021-04-01', '2021-05-01', '2021-06-01', '2021-07-01', '2021-08-01', '2021-09-01', '2021-10-01', '2021-11-01', '2021-12-01', '2022-01-01', '2022-02-01', '2022-03-01', '2022-04-01', '2022-05-01', '2022-06-01', '2022-07-01', '2022-08-01', '2022-09-01', '2022-10-01', '2022-11-01', '2022-12-01', '2023-01-01', '2023-02-01', '2023-03-01', '2023-04-01', '2023-05-01', '2023-06-01', '2023-07-01', '2023-08-01', '2023-09-01', '2023-10-01', '2023-11-01', '2023-12-01', '2024-01-01', '2024-02-01']


In [82]:
# Get the url of the tile layer from GEE.
from geemap.ee_tile_layers import _get_tile_url_format

basemap=nicfi.filter(ee.Filter.date('2019-01-01','2023-03-11')).sort('system:time_start', False).first()

print(type(basemap))
# Define visualization parameters
vis={'bands':['R','G','B'],'min':500,'max':3500,'gamma':2.0}

# Get the tile URL format
map_id_dict = ee.Image(basemap).getMapId(vis)
print(map_id_dict)
url = map_id_dict["tile_fetcher"].url_format
print(url)

url_format = _get_tile_url_format(basemap, vis)
print(url_format)

<class 'ee.image.Image'>
{'mapid': 'projects/ee-qinheyi/maps/a8747266b8de80cffe594a0c078b3b4c-4115f79a30eba797b6c398402fa88132', 'token': '', 'tile_fetcher': <ee.data.TileFetcher object at 0x000001F39CC21070>, 'image': <ee.image.Image object at 0x000001F39CC4F4F0>}
https://earthengine.googleapis.com/v1/projects/ee-qinheyi/maps/a8747266b8de80cffe594a0c078b3b4c-4115f79a30eba797b6c398402fa88132/tiles/{z}/{x}/{y}
https://earthengine.googleapis.com/v1/projects/ee-qinheyi/maps/a8747266b8de80cffe594a0c078b3b4c-e42174e0132e6abec57d4116f009c856/tiles/{z}/{x}/{y}
